# Hệ thống gợi ý phim với TMDB 5000

**Môn**: Final Project - Recommendation System  
**Mô tả**: Xây dựng hệ thống gợi ý phim (movie recommender) dựa trên dữ liệu TMDB 5000 (`tmdb_5000_movies.csv`, `tmdb_5000_credits.csv`).

Các phần chính:
1. Thu thập & nạp dữ liệu
2. Làm sạch & chuẩn bị dữ liệu
3. Phân tích & trực quan hóa dữ liệu
4. Xây dựng hệ gợi ý (content-based)
5. Đánh giá mô hình (RMSE, MAE, Precision@K, Recall@K)
6. Giao diện gợi ý trong notebook (nhập tên phim để nhận gợi ý)



In [ ]:
# Cài đặt và import thư viện

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

pd.set_option('display.max_colwidth', 200)
plt.style.use('seaborn-v0_8')

print("Thư viện đã được import.")


In [ ]:
# 1. Thu thập & nạp dữ liệu

movies_path = 'tmdb_5000_movies.csv'
credits_path = 'tmdb_5000_credits.csv'

movies = pd.read_csv(movies_path)
credits = pd.read_csv(credits_path)

print('Kích thước movies:', movies.shape)
print('Kích thước credits:', credits.shape)

movies.head(3)


In [ ]:
# 2. Làm sạch & chuẩn bị dữ liệu

# Gộp thông tin credits vào movies theo movie_id
credits_renamed = credits.rename(columns={'movie_id': 'id'})
movies_merged = movies.merge(credits_renamed[['id', 'cast', 'crew']], on='id', how='left')

print('Kích thước sau khi merge:', movies_merged.shape)

# Loại bỏ duplicate theo title
before_dups = movies_merged.shape[0]
movies_merged = movies_merged.drop_duplicates(subset=['title'])
after_dups = movies_merged.shape[0]
print(f'Drop duplicate theo title: {before_dups} -> {after_dups}')

# Xử lý missing values cho text: thay bằng chuỗi rỗng
text_cols = ['overview', 'tagline', 'cast', 'crew', 'keywords', 'genres']
for col in text_cols:
    if col in movies_merged.columns:
        movies_merged[col] = movies_merged[col].fillna('')

# Xử lý missing cho numeric: thay bằng median
num_cols = ['vote_average', 'vote_count', 'popularity', 'runtime']
for col in num_cols:
    if col in movies_merged.columns:
        movies_merged[col] = movies_merged[col].fillna(movies_merged[col].median())

# Xử lý outlier đơn giản: clip vote_count ở percentiles 1% - 99%
low, high = movies_merged['vote_count'].quantile([0.01, 0.99])
movies_merged['vote_count_clipped'] = movies_merged['vote_count'].clip(lower=low, upper=high)

# Chuẩn hóa một số đặc trưng numeric
scaler = MinMaxScaler()
movies_merged[['vote_avg_scaled', 'popularity_scaled', 'vote_count_scaled']] = scaler.fit_transform(
    movies_merged[['vote_average', 'popularity', 'vote_count_clipped']]
)

movies_merged[['title', 'vote_average', 'vote_avg_scaled']].head(3)


## 3. Phân tích & Trực quan hóa dữ liệu


In [ ]:
### 3.1. Phân bố Rating (vote_average)

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax[0].hist(movies_merged['vote_average'], bins=30, edgecolor='black', alpha=0.7)
ax[0].set_xlabel('Vote Average')
ax[0].set_ylabel('Frequency')
ax[0].set_title('Phân bố điểm đánh giá phim')
ax[0].grid(alpha=0.3)

# Boxplot
ax[1].boxplot(movies_merged['vote_average'], vert=True)
ax[1].set_ylabel('Vote Average')
ax[1].set_title('Boxplot của Vote Average')
ax[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean vote: {movies_merged['vote_average'].mean():.2f}")
print(f"Median vote: {movies_merged['vote_average'].median():.2f}")
print(f"Std vote: {movies_merged['vote_average'].std():.2f}")


In [ ]:
### 3.2. Tần suất thể loại phim (Genres)

import json
from collections import Counter

# Parse genres từ JSON string
def extract_genres(genres_str):
    try:
        genres_list = json.loads(genres_str)
        return [g['name'] for g in genres_list]
    except:
        return []

movies_merged['genres_list'] = movies_merged['genres'].apply(extract_genres)

# Đếm tần suất
all_genres = []
for genres in movies_merged['genres_list']:
    all_genres.extend(genres)

genre_counts = Counter(all_genres)
top_genres = genre_counts.most_common(15)

# Vẽ bar chart
genres_df = pd.DataFrame(top_genres, columns=['Genre', 'Count'])
plt.figure(figsize=(12, 6))
plt.barh(genres_df['Genre'], genres_df['Count'], color='steelblue')
plt.xlabel('Số lượng phim')
plt.ylabel('Thể loại')
plt.title('Top 15 thể loại phim phổ biến nhất')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Tổng số thể loại khác nhau: {len(genre_counts)}")


In [ ]:
### 3.3. Top 10 phim có rating cao nhất

top_rated = movies_merged.nlargest(10, 'vote_average')[['title', 'vote_average', 'vote_count', 'popularity']]
print(top_rated.to_string(index=False))


In [ ]:
### 3.4. Heatmap tương quan giữa các biến số

corr_cols = ['vote_average', 'vote_count', 'popularity', 'runtime', 'budget', 'revenue']
corr_data = movies_merged[corr_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_data, annot=True, fmt='.2f', cmap='coolwarm', square=True, linewidths=0.5)
plt.title('Heatmap tương quan giữa các biến số')
plt.tight_layout()
plt.show()


## 4. Xây dựng hệ gợi ý phim (Content-Based Filtering)


In [ ]:
### 4.1. Chuẩn bị features cho mô hình

# Hàm parse keywords
def extract_keywords(keywords_str):
    try:
        keywords_list = json.loads(keywords_str)
        return ' '.join([k['name'] for k in keywords_list])
    except:
        return ''

# Hàm parse cast (lấy 5 diễn viên đầu tiên)
def extract_cast(cast_str):
    try:
        cast_list = json.loads(cast_str)
        return ' '.join([c['name'].replace(' ', '') for c in cast_list[:5]])
    except:
        return ''

# Hàm parse director từ crew
def extract_director(crew_str):
    try:
        crew_list = json.loads(crew_str)
        for person in crew_list:
            if person.get('job') == 'Director':
                return person['name'].replace(' ', '')
        return ''
    except:
        return ''

movies_merged['keywords_clean'] = movies_merged['keywords'].apply(extract_keywords)
movies_merged['cast_clean'] = movies_merged['cast'].apply(extract_cast)
movies_merged['director_clean'] = movies_merged['crew'].apply(extract_director)
movies_merged['genres_clean'] = movies_merged['genres_list'].apply(lambda x: ' '.join([g.replace(' ', '') for g in x]))

# Kết hợp các features thành một chuỗi duy nhất
movies_merged['combined_features'] = (
    movies_merged['overview'].fillna('') + ' ' +
    movies_merged['genres_clean'] + ' ' +
    movies_merged['keywords_clean'] + ' ' +
    movies_merged['cast_clean'] + ' ' +
    movies_merged['director_clean']
)

print("Sample combined features:")
print(movies_merged[['title', 'combined_features']].head(2))


In [ ]:
### 4.2. Vector hóa với TF-IDF và tính Cosine Similarity

# Khởi tạo TF-IDF Vectorizer
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)
)

# Fit và transform
tfidf_matrix = tfidf.fit_transform(movies_merged['combined_features'])

print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")

# Tính cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(f"Cosine similarity matrix shape: {cosine_sim.shape}")

# Tạo mapping từ title sang index
indices = pd.Series(movies_merged.index, index=movies_merged['title']).drop_duplicates()

print(f"\\nSố phim trong hệ thống: {len(indices)}")


In [ ]:
### 4.3. Hàm gợi ý phim

def get_recommendations(title, top_n=10):
    """
    Trả về top N phim tương tự với phim có title đã cho
    """
    try:
        # Lấy index của phim
        idx = indices[title]
        
        # Lấy similarity scores
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sắp xếp theo similarity (giảm dần)
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Lấy top N phim (bỏ phim đầu tiên vì chính nó)
        sim_scores = sim_scores[1:top_n+1]
        
        # Lấy indices của phim
        movie_indices = [i[0] for i in sim_scores]
        
        # Trả về thông tin phim
        result = movies_merged.iloc[movie_indices][['title', 'vote_average', 'vote_count', 'genres_clean', 'overview']]
        result['similarity_score'] = [score[1] for score in sim_scores]
        
        return result
    except KeyError:
        return f"Phim '{title}' không tồn tại trong cơ sở dữ liệu."

# Test hàm gợi ý
print("=== Gợi ý phim tương tự 'Avatar' ===")
recommendations = get_recommendations('Avatar', top_n=5)
print(recommendations[['title', 'similarity_score', 'vote_average']].to_string(index=False))


## 5. Đánh giá mô hình


In [ ]:
### 5.1. Đánh giá RMSE và MAE

# Với content-based filtering, ta đánh giá bằng cách:
# - Dự đoán rating của phim gợi ý = trung bình có trọng số theo similarity
# - So sánh với rating thực tế

def predict_rating(title, top_n=10):
    """Dự đoán rating cho phim dựa trên các phim tương tự"""
    try:
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
        
        # Tính rating dự đoán = weighted average
        total_sim = sum([score[1] for score in sim_scores])
        if total_sim == 0:
            return movies_merged.iloc[idx]['vote_average']
        
        weighted_rating = sum([
            movies_merged.iloc[score[0]]['vote_average'] * score[1] 
            for score in sim_scores
        ]) / total_sim
        
        return weighted_rating
    except:
        return None

# Lấy sample để đánh giá (100 phim có vote_count > 100)
sample_movies = movies_merged[movies_merged['vote_count'] > 100].sample(min(100, len(movies_merged)), random_state=42)

y_true = []
y_pred = []

for title in sample_movies['title']:
    true_rating = sample_movies[sample_movies['title'] == title]['vote_average'].values[0]
    pred_rating = predict_rating(title, top_n=10)
    
    if pred_rating is not None:
        y_true.append(true_rating)
        y_pred.append(pred_rating)

# Tính RMSE và MAE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"\\nSố phim đánh giá: {len(y_true)}")


In [ ]:
### 5.2. Đánh giá Precision@K và Recall@K

# Giả định: Phim được coi là "relevant" nếu có rating >= threshold và cùng thể loại
def evaluate_precision_recall(test_movies, K=10, rating_threshold=7.0):
    """
    Đánh giá Precision@K và Recall@K
    - Relevant items: phim có rating >= threshold và chia sẻ ít nhất 1 thể loại với phim gốc
    """
    precisions = []
    recalls = []
    
    for title in test_movies['title']:
        movie_info = movies_merged[movies_merged['title'] == title].iloc[0]
        movie_genres = set(movie_info['genres_list'])
        
        # Lấy recommendations
        recs = get_recommendations(title, top_n=K)
        
        if isinstance(recs, str):  # Trường hợp không tìm thấy
            continue
        
        # Tìm relevant items trong toàn bộ dataset
        relevant_items = movies_merged[
            (movies_merged['vote_average'] >= rating_threshold) &
            (movies_merged['title'] != title) &
            (movies_merged['genres_list'].apply(lambda x: len(set(x) & movie_genres) > 0))
        ]['title'].tolist()
        
        if len(relevant_items) == 0:
            continue
        
        # Tính số recommended items là relevant
        recommended_titles = recs['title'].tolist()
        relevant_recommended = set(recommended_titles) & set(relevant_items)
        
        # Precision@K = (relevant items in top K) / K
        precision = len(relevant_recommended) / K
        
        # Recall@K = (relevant items in top K) / (total relevant items)
        recall = len(relevant_recommended) / len(relevant_items) if len(relevant_items) > 0 else 0
        
        precisions.append(precision)
        recalls.append(recall)
    
    return np.mean(precisions), np.mean(recalls)

# Đánh giá trên sample
test_sample = movies_merged[movies_merged['vote_count'] > 50].sample(min(50, len(movies_merged)), random_state=42)
precision_at_10, recall_at_10 = evaluate_precision_recall(test_sample, K=10, rating_threshold=7.0)

print(f"Precision@10: {precision_at_10:.4f}")
print(f"Recall@10: {recall_at_10:.4f}")
print(f"\\nSố phim test: {len(test_sample)}")
print(f"Ngưỡng rating cho relevant items: 7.0")


In [ ]:
### 5.3. Tóm tắt kết quả đánh giá

print("=" * 60)
print("TỔNG KẾT KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH")
print("=" * 60)
print(f"✓ Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"✓ Mean Absolute Error (MAE): {mae:.4f}")
print(f"✓ Precision@10: {precision_at_10:.4f}")
print(f"✓ Recall@10: {recall_at_10:.4f}")
print("=" * 60)
print(f"\\nMô hình content-based filtering đã được xây dựng thành công!")
print(f"Dataset: {len(movies_merged)} phim từ TMDB 5000")
print(f"Features: overview, genres, keywords, cast, director")
print(f"Vectorization: TF-IDF (max_features=5000, ngram_range=(1,2))")


In [ ]:
## 6. Lưu model và dữ liệu để triển khai

import pickle

# Lưu các object cần thiết
data_to_save = {
    'movies_data': movies_merged[['id', 'title', 'vote_average', 'vote_count', 'popularity', 
                                   'genres_clean', 'overview', 'release_date', 'runtime']],
    'cosine_sim': cosine_sim,
    'indices': indices
}

with open('movie_recommender_model.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

print("✓ Đã lưu model và dữ liệu vào file 'movie_recommender_model.pkl'")
print(f"✓ Kích thước file: {os.path.getsize('movie_recommender_model.pkl') / (1024*1024):.2f} MB")


## 7. Hướng dẫn sử dụng Web App

Để chạy giao diện web Streamlit:

1. **Đảm bảo đã chạy tất cả cell ở trên** để tạo file `movie_recommender_model.pkl`

2. **Mở terminal/command prompt** và chạy lệnh:
   ```
   streamlit run app.py
   ```

3. **Trình duyệt sẽ tự động mở** tại `http://localhost:8501`

4. **Sử dụng app**:
   - Chọn phim yêu thích từ dropdown
   - Điều chỉnh số lượng gợi ý
   - Nhấn nút "Tìm phim tương tự"
   - Xem kết quả và download CSV nếu cần

---

### 🎉 Hoàn thành!

Project đã bao gồm:
- ✅ Notebook phân tích đầy đủ (`tmdb_recommender.ipynb`)
- ✅ Web App với Streamlit (`app.py`)
- ✅ Requirements file (`requirements.txt`)
- ✅ Hướng dẫn chi tiết (`README.md`)
- ✅ Model đã lưu (`movie_recommender_model.pkl`)

Chúc bạn hoàn thành tốt Final Project! 🚀
